In [1]:
import requests

def search_episuite(query_term):
    url = 'https://episuite.dev/EpiWebSuite/api/search'
    params = {'query': query_term}
    
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for HTTP errors
    data = response.json()
    return data



query_term = '50-00-0'  # Replace with your query term
result = search_episuite(query_term)
print(result)

[{'name': 'FORMALDEHYDE', 'smiles': 'O=C', 'cas': '000050-00-0'}]


In [2]:
url = 'https://episuite.dev/EpiWebSuite/api/search'
params = {'query': query_term}

response = requests.get(url, params=params)
response.raise_for_status()  # Raise an exception for HTTP errors
data = response.json()

In [3]:
response.status_code
response.headers

{'Date': 'Mon, 04 Nov 2024 21:29:18 GMT', 'Content-Type': 'application/json', 'Content-Length': '89', 'Connection': 'keep-alive', 'Server': 'Apache', 'access-control-allow-origin': '*', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Referrer-Policy': 'same-origin', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'Cache-Control': 'max-age=0, no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload'}

In [4]:
from pyepisuite.api_client import EpiSuiteAPIClient
epi = EpiSuiteAPIClient()
ids = epi.search('50-00-0')
res_cas = epi.submit(cas= ids[0].cas)
# res_name = epi.submit(smiles= ids[0].smiles)

In [5]:
def from_dict(data_class, data):
    """
    Recursively converts a dictionary to a dataclass instance.
    """
    if isinstance(data, list):
        return [from_dict(data_class.__args__[0], item) for item in data]
    elif isinstance(data, dict):
        fieldtypes = {f.name: f.type for f in data_class.__dataclass_fields__.values()}
        return data_class(**{f: from_dict(fieldtypes[f], data[f]) if isinstance(fieldtypes[f], type) else data[f] for f in data})
    else:
        return data

In [6]:
from pyepisuite.models import ResultEcoSAR, ResultEPISuite
res_ecosar = res_cas['ecosar']
# remove ecosar from the dictionary
res_cas.pop('ecosar', None)
# remove output and alerts from the dictionary
res_ecosar.pop('output', None)
# res_ecosar.pop('alerts', None)
ecosar_data  = from_dict(ResultEcoSAR, res_ecosar)

In [7]:
ecosar_data.parameters.logKow.value

0.35

In [8]:
ecosar_data

ResultEcoSAR(parameters=EcosarParameters(smiles='O=C', cas='000050-00-0', logKow=Parameter(value=0.35, units=None, valueType='EXPERIMENTAL'), waterSolubility=Parameter(value=54982.12109375, units='mg/L', valueType='ESTIMATED'), meltingPoint=Parameter(value=-92.0, units='celsius', valueType='EXPERIMENTAL')), modelResults=[{'qsarClass': 'Aldehydes (Mono)', 'organism': 'Fish', 'duration': '96-hr', 'endpoint': 'LC50', 'concentration': 11.226508140563965, 'maxLogKow': 5.0, 'flags': []}, {'qsarClass': 'Aldehydes (Mono)', 'organism': 'Daphnid', 'duration': '48-hr', 'endpoint': 'LC50', 'concentration': 11.985322952270508, 'maxLogKow': 5.0, 'flags': []}, {'qsarClass': 'Aldehydes (Mono)', 'organism': 'Green Algae', 'duration': '96-hr', 'endpoint': 'EC50', 'concentration': 5.865765571594238, 'maxLogKow': 6.400000095367432, 'flags': []}, {'qsarClass': 'Aldehydes (Mono)', 'organism': 'Fish', 'duration': '', 'endpoint': 'ChV', 'concentration': 3.619093179702759, 'maxLogKow': 8.0, 'flags': []}, {'qsa

In [9]:
from typing import Optional
def from_dict_main(data_class, data):
    if isinstance(data_class, type):
        if hasattr(data_class, '__dataclass_fields__'):
            fieldtypes = {f.name: f.type for f in data_class.__dataclass_fields__.values()}
            print(fieldtypes)
            return data_class(**{f: from_dict_main(fieldtypes[f], data[f]) for f in data})
    elif hasattr(data_class, '__origin__'):
        origin = data_class.__origin__
        if origin is list:
            return [from_dict_main(data_class.__args__[0], item) for item in data]
        elif origin is Optional:
            return from_dict_main(data_class.__args__[0], data) if data is not None else None
    return data

In [35]:
res_epi = from_dict_main(ResultEPISuite, res_cas)
# res_epi = from_dict(ResultEPISuite, res_cas)

{'parameters': <class 'pyepisuite.models.Parameters'>, 'chemicalProperties': <class 'pyepisuite.models.ChemicalProperties'>, 'logKow': <class 'pyepisuite.models.LogKowResponse'>, 'meltingPoint': <class 'pyepisuite.models.MeltingPointResponse'>, 'boilingPoint': <class 'pyepisuite.models.BoilingPointResponse'>, 'vaporPressure': <class 'pyepisuite.models.VaporPressureResponse'>, 'waterSolubilityFromLogKow': <class 'pyepisuite.models.WaterSolubilityFromLogKowResponse'>, 'waterSolubilityFromWaterNt': <class 'pyepisuite.models.WaterSolubilityFromWaterNtResponse'>, 'henrysLawConstant': <class 'pyepisuite.models.HenrysLawConstantResponse'>, 'logKoa': <class 'pyepisuite.models.LogKoaResponse'>, 'biodegradationRate': <class 'pyepisuite.models.BiodegradationRateResponse'>, 'hydrocarbonBiodegradationRate': <class 'pyepisuite.models.HydrocarbonBiodegradationRateResponse'>, 'aerosolAdsorptionFraction': <class 'pyepisuite.models.AerosolAdsorptionFractionResponse'>, 'atmosphericHalfLife': <class 'pyep

KeyError: 'parameters'